In [1]:
import os
import tqdm as tqdm
import numpy as np
import copy

# 处理服务器中evo的可视化问题
import evo
from evo.tools.settings import SETTINGS
SETTINGS['plot_backend'] = 'Agg'

from evo.tools import file_interface, plot
from evo.core.geometry import GeometryException
import evo.main_ape as main_ape
from evo.core import sync, metrics
from evo.core.trajectory import PoseTrajectory3D

import matplotlib.pyplot as plt #绘图

print("Successfully import ultils")

Successfully import ultils


In [2]:
def make_evo_traj_gt(poses_N_x_7, tss_us):
    assert poses_N_x_7.shape[1] == 7
    assert poses_N_x_7.shape[0] > 10
    assert tss_us.shape[0] == poses_N_x_7.shape[0]

    traj_evo = PoseTrajectory3D(
        positions_xyz=poses_N_x_7[:,:3],
        # orientations_quat_wxyz=poses_N_x_7[:,3:],
        orientations_quat_wxyz = poses_N_x_7[:, [6,3,4,5]],#gt存储的是xyzw
        timestamps=tss_us/1e6)#转换为秒
    return traj_evo

def make_evo_traj_deio(poses_N_x_7, tss_us):
    assert poses_N_x_7.shape[1] == 7
    assert poses_N_x_7.shape[0] > 10
    assert tss_us.shape[0] == poses_N_x_7.shape[0]

    traj_evo = PoseTrajectory3D(
        positions_xyz=poses_N_x_7[:,:3],
        # orientations_quat_wxyz=poses_N_x_7[:,3:],
        orientations_quat_wxyz = poses_N_x_7[:, [5,6,3,4]],#存储的是yzwx(由于原代码的bug导致的,注意统一代码输出为xyzw而evo中需要用的是wxyz即可)
        timestamps=tss_us/1e6)#转换为秒
    return traj_evo

def load_gt_us(path, skiprows=0):
    traj_ref = np.loadtxt(path, delimiter=" ", skiprows=skiprows)
    tss_gt_us = traj_ref[:, 0].copy() 
    assert np.all(tss_gt_us == sorted(tss_gt_us))
    assert traj_ref.shape[0] > 0
    assert traj_ref.shape[1] == 8

    return tss_gt_us, traj_ref[:, 1:]

def plot_trajectory_inxyplane(pred_traj, gt_traj, align=True, _n_to_align=-1,correct_scale=True, max_diff_sec=1.0,title="", filename=""):
    #将两个轨迹对齐(时间维度上的)
    gt_traj, pred_traj = sync.associate_trajectories(gt_traj, pred_traj, max_diff=max_diff_sec)

    # 对齐轨迹(空间维度上的)
    if align:
        try:
            pred_traj.align(gt_traj, correct_scale=correct_scale,n=_n_to_align)
        except GeometryException as e:
            print("Plotting error:", e)

    plot_collection = plot.PlotCollection("PlotCol")

    fig = plt.figure(figsize=(8, 8))
    plot_mode = plot.PlotMode.xy
    ax = plot.prepare_axis(fig, plot_mode)
    ax.set_title(title)
    if gt_traj is not None:
        plot.traj(ax, plot_mode, gt_traj, '--', 'gray', "Ground Truth")
    plot.traj(ax, plot_mode, pred_traj, '-', 'blue', "Predicted")
    
    plot_collection.add_figure("traj (error)", fig)

    plt.show()

print("Successfully define several functions")

Successfully define several functions


In [3]:
print("Evaluation for Stereo-HKU dataset")
indir="/media/lfl-data2/Stereo_HKU/"# your dataset path
cur_dir=os.getcwd() #current file directoin

target_dirs = {
                "HKU_aggressive_translation",
                "HKU_aggressive_rotation",
                "HKU_aggressive_small_flip",
                "hku_aggressive_walk",
                "HKU_HDR_circle",
                "HKU_HDR_slow",
                "hku_hdr_tran_rota",
                "hku_hdr_agg",
                "hku_dark_normal",
                }

for root, dirs, files in os.walk(indir):
    for d in dirs:
        # 构建完整路径 data_path
        datapath_val = os.path.join(root, d)

        # 检查是否为目标文件夹之一
        if os.path.basename(datapath_val) in target_dirs:
            sequence_name = os.path.basename(datapath_val)

            # 获取轨迹
            tss_gt_us, traj_gt = load_gt_us(os.path.join(datapath_val, f"raw_gt_stamped_us.txt"))#获取真实轨迹

            # 获取deio估算的轨迹
            if sequence_name == "HKU_aggressive_translation":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./Stereo-HKU/2024-10-09_HKU_aggressive_translation/Hku_Aggressive_Translation_Trial01.txt"))
            elif sequence_name == "HKU_aggressive_rotation":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./Stereo-HKU/2024-10-09_HKU_aggressive_rotation/Hku_Aggressive_Rotation_Trial01.txt"))
            elif sequence_name == "HKU_aggressive_small_flip":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./Stereo-HKU/2024-10-09_HKU_aggressive_small_flip/Hku_Aggressive_Small_Flip_Trial01.txt"))
            elif sequence_name == "hku_aggressive_walk":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./Stereo-HKU/2024-10-10_hku_aggressive_walk/Hku_Aggressive_Walk_Trial01.txt"))
            elif sequence_name == "HKU_HDR_circle":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./Stereo-HKU/2024-10-09_HKU_HDR_circle/Hku_Hdr_Circle_Trial01.txt"))
            elif sequence_name == "HKU_HDR_slow":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./Stereo-HKU/2024-10-09_HKU_HDR_slow/Hku_Hdr_Slow_Trial01.txt"))
            elif sequence_name == "hku_hdr_tran_rota":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./Stereo-HKU/2024-10-09_hku_hdr_tran_rota/Hku_Hdr_Tran_Rota_Trial01.txt"))
            elif sequence_name == "hku_hdr_agg":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./Stereo-HKU/2024-10-09_hku_hdr_agg/Hku_Hdr_Agg_Trial01.txt"))
            elif sequence_name == "hku_dark_normal":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./Stereo-HKU/2024-10-09_hku_dark_normal/Hku_Dark_Normal_Trial01.txt"))

            evoGT = make_evo_traj_gt(traj_gt, tss_gt_us)
            evoEst = make_evo_traj_deio(traj_deio, tss_deio_us)
            gtlentraj = evoGT.get_infos()["path length (m)"]#获取轨迹长度
            evoGT, evoEst = sync.associate_trajectories(evoGT, evoEst, max_diff=1)
            _n_to_align=-1;
            ape_trans = main_ape.ape(copy.deepcopy(evoGT), copy.deepcopy(evoEst), pose_relation=metrics.PoseRelation.translation_part, align=True,n_to_align=_n_to_align, correct_scale=True)

            # print(f"\033[31m EVO结果：{ape_trans}\033[0m");
            MPE = ape_trans.stats["mean"] / gtlentraj * 100
            # print(f"MPE is {MPE:.02f}") #注意只保留两位小数
            evoATE = ape_trans.stats["rmse"]*100

            res_str = f"\nATE[cm]: {evoATE:.02f} | MPE[%/m]: {MPE:.02f}"
            
            print(f"{sequence_name}: {res_str}")
            
    # 使用break限制os.walk只遍历indir的第一层
    break

Evaluation for Stereo-HKU dataset
HKU_aggressive_rotation: 
ATE[cm]: 6.45 | MPE[%/m]: 0.09
HKU_aggressive_small_flip: 
ATE[cm]: 9.86 | MPE[%/m]: 0.20
HKU_aggressive_translation: 
ATE[cm]: 4.37 | MPE[%/m]: 0.06
hku_aggressive_walk: 
ATE[cm]: 45.71 | MPE[%/m]: 0.48
hku_dark_normal: 
ATE[cm]: 10.75 | MPE[%/m]: 0.11
hku_hdr_agg: 
ATE[cm]: 6.68 | MPE[%/m]: 0.06
HKU_HDR_circle: 
ATE[cm]: 8.17 | MPE[%/m]: 0.14
HKU_HDR_slow: 
ATE[cm]: 4.38 | MPE[%/m]: 0.07
hku_hdr_tran_rota: 
ATE[cm]: 6.37 | MPE[%/m]: 0.09
